# **Installing libraries**

In [ ]:
#@title Installing libraries and fetching some scripts
%%capture
! pip install ase chemicals pubchempy
! pip install --upgrade x3dase
! sudo apt install cm-super dvipng texlive-latex-extra texlive-latex-recommended
! curl -LJO https://raw.githubusercontent.com/emartineznunez/Master_Quimica/master/.local/bin/mopac
! curl -LJO https://raw.githubusercontent.com/emartineznunez/Master_Quimica/master/lib/libiomp5.so
! curl -LJO https://github.com/emartineznunez/AutoMeKin/raw/main/scripts/get_geom_mopac.sh
! curl -LJO https://github.com/emartineznunez/AutoMeKin/raw/main/scripts/get_freq_mopac.sh
%env PATH="/content:.:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin"
%env LD_LIBRARY_PATH="/content:/usr/local/nvidia/lib:/usr/local/nvidia/lib64"
!chmod +x mopac thermochem.py get_geom_mopac.sh get_freq_mopac.sh

# **Selecting the two substances**

In [ ]:
#@title Retrieving the CIDs of the substances from PubChem
import pubchempy as pcp
import sys

nm = input('Identifier type: cid, name, smiles, sdf, inchi, inchikey, or formula: ')
if nm not in ['cid','name','smiles','sdf','inchi','inchikey','formula']:
  print('Please select one of the identifiers indicated below')
  sys.exit()

molA = input('Type your first molecule here: ')

queryA = pcp.get_compounds(molA,nm,record_type='3d')
cid_A = pcp.get_cids(molA)
if len(cid_A) == 0:
  print('First molecule not found')
else:
  print('CID of molecule A: ',cid_A)

molB = input('Type your second molecule here: ')
queryB = pcp.get_compounds(molB,nm,record_type='3d')
cid_B = pcp.get_cids(molB)
if len(cid_B) == 0:
  print('Second molecule not found')
else:
  print('CID of molecule B: ',cid_B)

# **Optimizations with MOPAC**

In [ ]:
#@title Molecule A
from x3dase.visualize import view_x3d_n
from ase.io import read
import IPython

m = queryA[0]
fA = open('molA.mop','w')
fA.write(str('precise denout' + '\n title \n title \n'))
for i,a in enumerate(m.atoms): fA.write(m.elements[i]+' '+str(a.x)+' '+str(a.y)+' '+str(a.z)+'\n')
fA.write('\noldgeo oldens force thermo(298,298)')
fA.close()
!mopac molA.mop &> molA.log
ene_A = ! awk '/FINAL HEAT OF FORMATION/{print $6}' molA.out
zpe_A = ! awk '/ZERO POINT ENERGY/{print $4}' molA.out
sigmaA = ! awk '/SYMMETRY NUMBER/{print $NF}' molA.out

print('Energy of molecule A: %10.2f kcal/mol' % float(ene_A[0]))
print('ZPVE   of molecule A: %10.2f kcal/mol' % float(zpe_A[0]))
print('')
print('')
print('VISUALIZATION OF THE OPTIMIZED STRUCTURE')

!get_geom_mopac.sh molA.out > molA.xyz
atoms = read('molA.xyz')
view_x3d_n(atoms,output='molA.html', bond=1.0, label=True)
IPython.display.HTML(filename='molA.html')

In [ ]:
#@title Molecule B
m = queryB[0]
fB = open('molB.mop','w')
fB.write(str('precise denout' + '\n title \n title \n'))
for i,a in enumerate(m.atoms): fB.write(m.elements[i]+' '+str(a.x)+' '+str(a.y)+' '+str(a.z)+'\n')
fB.write('\noldgeo oldens force thermo(298,298)')
fB.close()
!mopac molB.mop &> molB.log
ene_B = ! awk '/FINAL HEAT OF FORMATION/{print $6}' molB.out
zpe_B = ! awk '/ZERO POINT ENERGY/{print $4}' molB.out
sigmaB = ! awk '/SYMMETRY NUMBER/{print $NF}' molB.out

print('Energy of molecule B: %10.2f kcal/mol' % float(ene_B[0]))
print('ZPVE   of molecule B: %10.2f kcal/mol' % float(zpe_B[0]))
print('')
print('')
print('VISUALIZATION OF THE OPTIMIZED STRUCTURE')

!get_geom_mopac.sh molB.out > molB.xyz
atoms = read('molB.xyz')
view_x3d_n(atoms,output='molB.html', bond=1.0, label=True)
IPython.display.HTML(filename='molB.html')

# **Plotting L-V diagrams**

In [ ]:
#@title Importing libraries and defining functions
import numpy as np
import sys
from chemicals import *
#from requests import get
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

def pA(T):
  return p0 * np.exp(dHA/R * (1/TA0 - 1/T) )

def pB(T):
  return p0 * np.exp(dHB/R * (1/TB0 - 1/T) )

def xa_l(T):
  return (ptot - pB(T)) / (pA(T) - pB(T))

def xa_v(T):
  return xa_l(T) * pA(T) / (pB(T) +  (pA(T) - pB(T)) * xa_l(T))

def tcross(x):
  return (dHB + x * (dHA - dHB)) / (dHB / TB + x * (dHA / TA - dHB / TB))

#Chemical potentials Molar Gibbs energies G_m = Sum_i x_i * mu_i
def G(mu0,x,p,T):
  return mu0 + R * T * np.log(p * x / p0)
def Gm_l(xa,T):
  return xa * G(muA0,xa,pA(T),T) + (1 - xa) * G(muB0,1-xa,pB(T),T)
def Gm_g(xa,T):
  return xa * G(muA0,xa,ptot,T) + (1 - xa) * G(muB0,1-xa,ptot,T)
def Gm_lg(xa,T):
    alpha = (xa - xa_v(T)) / (xa_l(T) - xa_v(T))
    beta  = (xa - xa_l(T)) / (xa_v(T) - xa_l(T))
    muA = alpha * xa_l(T) * G(muA0,xa_l(T),pA(T),T) + beta * xa_v(T) * G(muA0,xa_v(T),ptot,T)
    muB = alpha * (1-xa_l(T)) * G(muB0,1-xa_l(T),pB(T),T) + beta * (1-xa_v(T)) * G(muB0,1-xa_v(T),ptot,T)
    return muA + muB

In [ ]:
#@title Plotting the phase diagrams
#Pressures, temperature range for the L-V phase diagram and constants
p0 = 1 # standard pressure of 1 bar
R = 8.314472e-3  # kJ/(K mol)

ptot = float( input('Choose the pressure (in bar) for the T-composition diagram: ') )
temp = float( input('Choose the temperature (in K) for the P-composition diagram: ') )

#Retrieve properties of molecules
mA = 'pubchem='+str(cid_A[0])
moleculeA = search_chemical(mA)
TA0  = Tb(moleculeA.CASs)
TAc = Tc(moleculeA.CASs)
PcA = Pc(moleculeA.CASs)
dHA = Riedel(TA0,TAc,PcA) * 1e-3
mB = 'pubchem='+str(cid_B[0])
TA = 1 / (1 / TA0 - R / dHA * np.log(ptot / p0) )

moleculeB = search_chemical(mB)
TB0  = Tb(moleculeB.CASs)
TBc = Tc(moleculeB.CASs)
PcB = Pc(moleculeB.CASs)
dHB = Riedel(TB0,TBc,PcB) * 1e-3
TB = 1 / (1 / TB0 - R / dHB * np.log(ptot / p0) )

part = pA(temp)*750.062
pbrt = pB(temp)*750.062
print('\033[1mSubstance A :\033[0m ',molA)
print('\033[1mProperties:\033[0m')
print('Heat of vaporization: %10.2f kJ/mol' % dHA)
print('Boiling point:        %10.2f K' % TA)
print('Vapor pressure (%3.0f K): %8.2f torr' % (temp,part))
print('')
print('\033[1mSubstance B :\033[0m ',molB)
print('\033[1mProperties:\033[0m')
print('Heat of vaporization: %10.2f kJ/mol' % dHB)
print('Boiling point:        %10.2f K' % TB)
print('Vapor pressure (%3.0f K): %8.2f torr' % (temp,pbrt))
print('')

T = np.linspace(TA,TB,100)

#Plot the L-V diagram (P vs xa at T=temp)
x = np.linspace(0.,1.,100)
plt.rcParams['text.usetex'] = True
plt.plot(x,x*pA(temp)*750.062,'--',color='blue',label="A")
plt.plot(x,(1-x)*pB(temp)*750.062,'--',color='red',label="B")
plt.plot(x,(x*pA(temp)+(1-x)*pB(temp))*750.062,'-',color='black',label="A+B")
plt.ylabel(r'$P(\mathrm{torr})$',fontsize=20)
plt.xlabel(r'$x_{A}$',fontsize=20)
plt.legend()
plt.title('Pressure of each substance vs composition at T='+str(temp)+' K', fontsize=17)
plt.xlim(0,1)
plt.ylim(bottom=0)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
plt.savefig('P_PA_PB.svg')
plt.show()

#Plot the L-V diagram (P vs xa at T=temp)
x = np.linspace(0.,1.,100)
plt.rcParams['text.usetex'] = True
plt.plot( x,(pA(temp)*pB(temp) / ( x*(pB(temp)-pA(temp)) + pA(temp)) )*750.062,'-',color='red',label="gas")
plt.plot(x,(x*pA(temp)+(1-x)*pB(temp))*750.062,'-',color='blue',label="liquid")
plt.ylabel(r'$P(\mathrm{torr})$',fontsize=20)
plt.xlabel(r'$z_{A}$',fontsize=20)
plt.legend()
plt.title('Pressure-composition phase diagram at T='+str(temp)+' K', fontsize=17)
plt.xlim(0,1)
plt.ylim(bottom=0)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
plt.savefig('LV_diagram_P.svg')
plt.show()

#Plot the L-V diagram (T vs z at P=1 bar)
plt.rcParams['text.usetex'] = True
plt.plot(xa_l(T),T,'-',color='blue',label="liquid")
plt.plot(xa_v(T),T,'-',color='red',label="gas")
#plt.plot(x,tcross(x),'--',color='black')
plt.ylabel(r'$T(\mathrm{K})$',fontsize=20)
plt.xlabel(r'$z_{A}$',fontsize=20)
plt.legend()
plt.title('Temperature-composition phase diagram at P='+str(ptot)+' bar',fontsize=17)
plt.xlim(0,1)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
plt.savefig('LV_diagram_T.svg')
plt.show()

# **Plotting Gibbs energies of the different phases for different temperatures**

# Statistical therodynamics calculation of the standard-state chemical potential (Molar Gibbs energy) of each substance $\mu_m^{0}$.
### Using the Rigid-Rotor/Harmonic-Oscillator (RR/HO) model, this can be obtained from:

### $\mu_m^{0}=E_{el}+ZPVE-RT\ln \frac{q_m^{0}}{N_A}$

### where $E_{el}$ and $ZPVE$ are the electronic energy and zero-point vibrational energy (both determined in the above MOPAC calculation), and $-RT\ln (q_m^{0}/N_{A})$ is a thermal correction with:

### $\frac{q_m^{0}}{N_A} =\frac{q_{tr,m}^{0}}{N_A} \times q_{rot} \times q_{vib} = \left( \frac{2\pi mk_B T}{h^{2}} \right)^{3/2} \frac{k_BT}{P^{0}} \times 1/\sigma \left( \frac{k_BT}{hc} \right)^{3/2} \left( \frac{\pi}{\tilde{A}\tilde{B}\tilde{C}}\right)\times \prod_{i=1}^{N_v}\frac{1}{1-\exp(-hc\tilde{\nu} /k_BT)}$


In [ ]:
#@title Thermal correction in $\mu_m^{0}$


def G_m_0(temp,m,A,B,C,sigma,*freq):
  import math
  q_t = (2*math.pi*m*m_u*k_B*temp/h/h)**1.5*k_B*temp/bar2Pa
  rA  = h * c * 100 * A / k_B
  rB  = h * c * 100 * B / k_B
  rC  = h * c * 100 * C / k_B
  q_r = 1 / sigma * math.sqrt(math.pi)*(temp**1.5/math.sqrt(rA*rB*rC))
  q_v = 1

  for ele in freq:
    rv  = h * c * 100 * float(ele) / k_B
    q_v = -q_v / math.expm1(-rv/temp)
  q = q_t * q_r * q_v
  return - R * temp * np.log(q)

def pfs(temp,m,A,B,C,sigma,*freq):
  import math
  q_t = (2*math.pi*m*m_u*k_B*temp/h/h)**1.5*k_B*temp/bar2Pa
  rA  = h * c * 100 * A / k_B
  rB  = h * c * 100 * B / k_B
  rC  = h * c * 100 * C / k_B
  q_r = 1 / sigma * math.sqrt(math.pi)*(temp**1.5/math.sqrt(rA*rB*rC))
  q_v = 1

  for ele in freq:
    rv  = h * c * 100 * float(ele) / k_B
    q_v = -q_v / math.expm1(-rv/temp)
  q = q_t * q_r * q_v
  return sigma,A,B,C,q_t,q_r,q_v

In [ ]:
#@title Determining Gibbs energies

m_u = 1.66053886e-27      # atomic mass constant in kg
h = 6.6260693e-34         # J s
k_B = 1.3806505e-23       # J/K
c = 2.99792558e8          # speed of light in m/s
bar2Pa = 1e5          # 1 bar to N/m**2

#Determine the crossover temperatures
z = 0.5
# 1: when two phases start to co-exist at z = 0.5
# 2: when the gas phase is more stable
diff = diff2 = 1e10
for ele in T:
  if abs(xa_l(ele) - z) < diff:
    diff = abs(xa_l(ele) - z)
    cot = ele
  if abs(xa_v(ele) - z) < diff2:
    diff2 = abs(xa_v(ele) - z)
    cot2 = ele

#Print the range of temperatures where the two phases co-exist
print('Temperature range (K) where L and G co-exist at this composition: %6.2f-%6.2f' % (cot,cot2))

#Number of points to plot the Molar Gibbs energies of the different phases
N = 100
# For the temperature dependence of mu0 check this:
# https://chem.libretexts.org/Bookshelves/Physical_and_Theoretical_Chemistry_Textbook_Maps/Thermodynamics_and_Chemical_Equilibrium_(Ellgen)/24%3A_Indistinguishable_Molecules_-_Statistical_Thermodynamics_of_Ideal_Gases/24.10%3A_The_Gibbs_Free_Energy_for_One_Mole_of_An_Ideal_Gas
T = np.linspace(cot,cot2,N)

# The standard-state chemical potentials of the substances in the gas phase (determine from Statistical Thermodynamics)
muA0 = np.empty(N)
muB0 = np.empty(N)

! get_freq_mopac.sh molA.out > dum
freqA = ! awk '{if($1<0) print -$1; else print $1}' dum
mA = ! awk '/MOLECULAR WEIGHT/{print $4;exit}' molA.out
AA = ! awk '/A =/{print $3; exit}' molA.out
BA = ! awk '/A =/{print $6; exit}' molA.out
CA = ! awk '/A =/{print $9; exit}' molA.out

! get_freq_mopac.sh molB.out > dum
freqB = ! awk '{if($1<0) print -$1; else print $1}' dum
mB = ! awk '/MOLECULAR WEIGHT/{print $4;exit}' molB.out
AB = ! awk '/A =/{print $3; exit}' molB.out
BB = ! awk '/A =/{print $6; exit}' molB.out
CB = ! awk '/A =/{print $9; exit}' molB.out

for Ti in T:
  index = np.where(T == Ti)[0][0]
  corrA = G_m_0(Ti,float(mA[0]),float(AA[0]),float(BA[0]),float(CA[0]),float(sigmaA[0]),*freqA)
  muA0i = (float(ene_A[0])+float(zpe_A[0]) )*4.184 + corrA
  corrB = G_m_0(Ti,float(mB[0]),float(AB[0]),float(BB[0]),float(CB[0]),float(sigmaB[0]),*freqB)
  muB0i = (float(ene_B[0])+float(zpe_B[0]) )*4.184 + corrB
  muA0[index] = muA0i
  muB0[index] = muB0i

#Plot the Gibbs energies of the different phases
plt.rcParams['text.usetex'] = True
plt.plot(T,Gm_l(z,T),'-',color='blue', label='L')
plt.plot(T,Gm_g(z,T),'-',color='red', label='G')
plt.plot(T,Gm_lg(z,T),'-',color='black', label='L+G')
plt.ylabel(r'$G_{m}(\mathrm{kJ/mol})$',fontsize=20)
plt.xlabel(r'$T(\mathrm{K})$',fontsize=20)
plt.legend()
plt.xlim(cot,cot2)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
plt.savefig('Gibbs_energies.svg')
plt.show()